In [51]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'omkar_proj.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

from django_pandas.io import read_frame

In [82]:
from warehouse.models import (
    Property,
    Company,
    Gala
)
from account.models import (
    Farmer,
    Rental,
    Investor,
    User
)
from contract.models import (
    Rental as ContractRental,
    Investor as ContractInvestor,
    Farmer as ContractFarmer
    
)
from django.db.models import (
    Value,
    OuterRef,
    Count,
    F,
    ExpressionWrapper,
    IntegerField,
    CharField,
    Q
    
    
)
from django.db.models.functions import Concat
from django_pandas.io import read_frame
from notification.models import (
    RentalNotification
)
from superadmin.firebaseManager import (
    sendPush
)
from service.models import (
LeaveGalaRequest
)

In [91]:
ContractRental.objects.filter(
                user_id=5
                ).prefetch_related(
                    'gala__user_gala_leave_request'
                    ).values("gala__uid","gala__gala_number").annotate(
                        total_leave_request = Count("gala__user_gala_leave_request"),
                        leave_request_status  = F("gala__user_gala_leave_request__status")
                    ).filter(Q(leave_request_status = "Pending")|Q(leave_request_status = "Reject")).values("gala__uid","gala__gala_number").distinct()

<PolymorphicQuerySet [{'gala__uid': UUID('16b2b643-811b-4828-9a84-d41e957fca3d'), 'gala__gala_number': '1'}]>

In [81]:
ContractRental.objects.filter(user_id=5).prefetch_related(
    'gala__user_gala_leave_request'
    ).values("gala__uid","gala__gala_number").annotate(
    total_leave_request = LeaveGalaRequest.objects.filter(gala__uid=OuterRef("gala__uid"))
)

AttributeError: 'QuerySet' object has no attribute 'name'

In [39]:
from datetime import date,datetime
from dateutil.relativedelta import relativedelta

In [40]:
current_date = date.today()
previous_date = date.today() + relativedelta(months=+3)

In [41]:
str(current_date)

'2023-03-09'

In [42]:
datetime.strptime(str(current_date),"%Y-%m-%d").strftime("%d %b,%Y")

'09 Mar,2023'

In [43]:
contract_rental = ContractRental.objects.filter(agreement_valid_end_date=previous_date).values("user_id","gala_id","gala__gala_number","gala__warehouse__property_name"
        )

In [44]:
contract_rental

<PolymorphicQuerySet [{'user_id': 5, 'gala_id': 7, 'gala__gala_number': 'OM3/1', 'gala__warehouse__property_name': 'Omkar Warehouse 3'}]>

In [50]:
current_date = date.today()
current_date_with_formatter = datetime.strptime(str(current_date),"%Y-%m-%d").strftime("%d %b,%Y")
next_date = date.today() + relativedelta(months=+3)
next_date_with_formatter = datetime.strptime(str(next_date),"%Y-%m-%d").strftime("%d %b,%Y")

contract_rental = ContractRental.objects.filter(agreement_valid_end_date=next_date).values(
            "user_id","gala_id","gala__gala_number","gala__warehouse__property_name"
        )
if contract_rental:
    rental_instance = [RentalNotification(
        rental_id = i['user_id'],
        gala_id = i['gala_id'],
        status = "Renew_Gala",
        message = f"Your gala {i['gala__gala_number']}({i['gala__warehouse__property_name']}) is about to expire on {next_date_with_formatter}! you can renew your contract by submitting a requesting for gala {i['gala__gala_number']}"
    ) for i in contract_rental]
    RentalNotification.objects.bulk_update_or_create(rental_instance,['rental_id','gala_id','message'],match_field=['rental_id','gala_id','message'])
    contract_instance = ContractRental.objects.filter(agreement_valid_end_date=next_date).values(
        "user_id","gala_id","gala__gala_number","gala__warehouse__property_name","user__fcmdevice__registration_id"
    )
    for instance in contract_instance:
        sendPush("Renew Contract",f"Your gala {instance['gala__gala_number']}({instance['gala__warehouse__property_name']}) is about to expire on {next_date_with_formatter}! you can renew your contract by submitting a requesting for gala {instance['gala__gala_number']}",["cTSP-ZadSgqdRUJJycpyJz:APA91bFrWdkzkpkx6Qdk_PKeFzt3mM0OVfj1YPNWDmfj7eJag4b33ErFjVYFTH90BPsQjB7MKTPpx1vIEYdH_nciKANPyzQvlg0lo2DLqDxk0RR7DsLqa6rmfKESQyZKN_1RbbbmfmlY"])

In [36]:
contract_rental

<PolymorphicQuerySet [{'user_id': 5, 'gala_id': 7, 'gala__gala_number': 'OM3/1', 'gala__warehouse__property_name': 'Omkar Warehouse 3'}]>

In [91]:
contract_rental = ContractRental.objects.all().values("user_id","gala_id","gala__gala_number","gala__warehouse__property_name","user__fcmdevice__registration_id")

In [94]:
for i in contract_rental:
    print(i)

{'user_id': 5, 'gala_id': 7, 'gala__gala_number': 'OM3/1', 'gala__warehouse__property_name': 'Omkar Warehouse 3', 'user__fcmdevice__registration_id': 'fi077xEaTxCrpq4a-d9WeM:APA91bEEAxO_AuUCdL8HO599CraEYq9LijpS3EbKBfW73jwhA0SApLgeMN8i_sI4ugAqSoph7ktL5LX4g3mZRiFIftURdCQFZtJFg9kH7u_S8kMCX7EiCJ0awG9zmDz1VNT89mQEM'}
{'user_id': 5, 'gala_id': 6, 'gala__gala_number': '12', 'gala__warehouse__property_name': 'Omkar Warehouse 2', 'user__fcmdevice__registration_id': 'fi077xEaTxCrpq4a-d9WeM:APA91bEEAxO_AuUCdL8HO599CraEYq9LijpS3EbKBfW73jwhA0SApLgeMN8i_sI4ugAqSoph7ktL5LX4g3mZRiFIftURdCQFZtJFg9kH7u_S8kMCX7EiCJ0awG9zmDz1VNT89mQEM'}
{'user_id': 5, 'gala_id': 2, 'gala__gala_number': '2', 'gala__warehouse__property_name': 'Omkar Warehouse 1', 'user__fcmdevice__registration_id': 'fi077xEaTxCrpq4a-d9WeM:APA91bEEAxO_AuUCdL8HO599CraEYq9LijpS3EbKBfW73jwhA0SApLgeMN8i_sI4ugAqSoph7ktL5LX4g3mZRiFIftURdCQFZtJFg9kH7u_S8kMCX7EiCJ0awG9zmDz1VNT89mQEM'}
{'user_id': 5, 'gala_id': 1, 'gala__gala_number': '1', 'gala__war

In [95]:
notify = RentalNotification.objects.all()

In [102]:
for i in notify:
    print(i.rental.fcmdevice)

AttributeError: 'Rental' object has no attribute 'fcmdevice'

In [92]:
df = read_frame(contract_rental)

In [93]:
df

,user_id,gala_id,gala__gala_number,gala__warehouse__property_name,user__fcmdevice__registration_id
0,Ayan Rental,7,OM3/1,Omkar Warehouse 3,fi077xEaTxCrpq4a-d9WeM:APA91bEEAxO_AuUCdL8HO59...
1,Ayan Rental,6,12,Omkar Warehouse 2,fi077xEaTxCrpq4a-d9WeM:APA91bEEAxO_AuUCdL8HO59...
2,Ayan Rental,2,2,Omkar Warehouse 1,fi077xEaTxCrpq4a-d9WeM:APA91bEEAxO_AuUCdL8HO59...
3,Ayan Rental,1,1,Omkar Warehouse 1,fi077xEaTxCrpq4a-d9WeM:APA91bEEAxO_AuUCdL8HO59...


In [37]:
contract_rental = ContractRental.objects.all()
if contract_rental:
    rental_instance = [RentalNotification(
        rental_id = i.user.id ,
        gala_id = i.gala.id,
        message = f"Your gala {i.gala.gala_number}({i.gala.warehouse.property_name}) is about to expire! you can renew your contract by submitting a requesting for gala {i.gala.gala_number}"
    ) for i in contract_rental]
#     RentalNotification.objects.bulk_update_or_create(rental_instance,['rental_id','gala_id','message'],match_field=['rental_id','gala_id','message'])
#     self.stdout.write("Rental Notification is updated!")

In [48]:
for i in rental_instance:
    print(i)
    print(i.message)

Ayan Sheikh
Your gala OM3/1(Omkar Warehouse 3) is about to expire! you can renew your contract by submitting a requesting for gala OM3/1
Ayan Sheikh
Your gala 12(Omkar Warehouse 2) is about to expire! you can renew your contract by submitting a requesting for gala 12
Ayan Sheikh
Your gala 2(Omkar Warehouse 1) is about to expire! you can renew your contract by submitting a requesting for gala 2
Ayan Sheikh
Your gala 1(Omkar Warehouse 1) is about to expire! you can renew your contract by submitting a requesting for gala 1


In [46]:
list(ContractRental.objects.all().values_list("id",flat=True))

[9, 8, 2, 1]

In [3]:
get_farmer_warehouses = Company.objects.get(
                name="Omkar"
            ).get_properties.filter(
                farmer_warehouse_detail__user__groups__name="Farmer"
            ).values(
                "uid", "property_name", "property_type", "property_survey_number", "address", "city", "state", "country"
            ).annotate(
                total_number_of_galas=Gala.objects.filter(
                    warehouse__uid=OuterRef("uid"), is_allotted_to_farmer=True, gala_rental_contract_detail__owner__groups__name="Farmer"
                ).values("warehouse__uid").annotate(
                    total_galas_allotted=Count("id")).values("total_galas_allotted"),
                    total_number_of_rentals=ContractRental.objects.filter(
                        gala__warehouse__uid=OuterRef("uid"),
                        owner__groups__name="Farmer",
                ).values("gala__warehouse__uid").annotate(
                    total_number_of_rentals=Count("id")).values("total_number_of_rentals"),
                farmer_name=ContractFarmer.objects.filter(
                    warehouse__uid=OuterRef("uid")).values("user__username"),
                farmer_uid=ContractFarmer.objects.filter(
                    warehouse__uid=OuterRef("uid")).values("user__user_uid")
            )

In [4]:
get_farmer_warehouses

<QuerySet [{'uid': UUID('89e8ebf7-9e21-4300-aa03-5a96f419e498'), 'property_name': 'Omkar Warehouse 2', 'property_type': 'COLD_STORAGE', 'property_survey_number': '2', 'address': '...', 'city': '...', 'state': 'Madha Pradesh', 'country': 'India', 'total_number_of_galas': 1, 'total_number_of_rentals': 1, 'farmer_name': 'Ayan Farmer', 'farmer_uid': UUID('88928457-d428-480d-a7aa-a9b256043521')}, {'uid': UUID('690385b0-0e94-445a-b448-a18781846984'), 'property_name': 'Omkar Warehouse 1', 'property_type': 'PEB', 'property_survey_number': '1', 'address': 'Atulya IT Park', 'city': 'Indore', 'state': 'Madha Pradesh', 'country': 'India', 'total_number_of_galas': 2, 'total_number_of_rentals': 2, 'farmer_name': 'Ayan Farmer', 'farmer_uid': UUID('88928457-d428-480d-a7aa-a9b256043521')}]>

In [5]:
get_farmer_wh = Company.objects.get(name = "Omkar"
                    ).get_properties.filter(farmer_warehouse_detail__user__groups__name="Farmer"
                        ).values("uid","property_name"
        ).annotate(
                total_gala_count = Gala.objects.filter(warehouse__uid=OuterRef("uid"),is_allotted_to_farmer=True
                        ).values("warehouse__uid").annotate(total_gala_count =Count("id")).values("total_gala_count"),
                total_number_of_rentals=ContractRental.objects.filter(
                        gala__warehouse__uid=OuterRef("uid"),
                        owner__groups__name="Farmer",
                            ).values("gala__warehouse__uid").annotate(
                                    total_number_of_rentals= Count("id")).values("total_number_of_rentals"),
                                    total_gala_allotted = Gala.objects.filter(warehouse__uid=OuterRef("uid"),
                                                            
                                        is_allotted_to_farmer=True,gala_rental_contract_detail__owner__groups__name="Farmer"
                                                            
                                                            
                                            ).values("warehouse__uid").annotate(total_gala_allotted=Count("id")).values("total_gala_allotted"),
                                    farmer_name=ContractFarmer.objects.filter(
                                                    warehouse__uid=OuterRef("uid")).values("user__username"),
                                    farmer_uid=ContractFarmer.objects.filter(
                                                    warehouse__uid=OuterRef("uid")).values("user__user_uid")
                                                )
                  














In [6]:
get_farmer_wh

<QuerySet [{'uid': UUID('d20e7390-a74b-4a83-b6be-2f8f88e42e02'), 'property_name': 'warehouse 1', 'total_gala_count': 2, 'total_number_of_rentals': None, 'total_gala_allotted': None, 'farmer_name': 'ayan', 'farmer_uid': UUID('5f82381f-d95b-4cff-b2b8-0077c6d1f104')}]>

In [7]:
# farmer_gala_qs = Property.objects.filter(company__name="Omkar",uid="d20e7390-a74b-4a83-b6be-2f8f88e42e02").values("property_name").get_gala.filter(is_allotted=False,is_allotted_to_farmer=True,is_allotted_to_rental=False)

In [8]:
# farmer_gala_qs

In [9]:
# get_farmer_galas_qs = Company.objects.get(name="Omkar").get_properties.get(uid= "d20e7390-a74b-4a83-b6be-2f8f88e42e02"
#                     ).get_gala.filter(is_allotted=False,is_allotted_to_farmer=True,is_allotted_to_rental=False)

In [10]:
# get_farmer_galas_qs

In [11]:
farmer_qs = Company.objects.get(name="Omkar").get_properties.all().values("property_name"
                ).annotate(total_warehouse = Property.objects.filter(uid= OuterRef("uid"),is_allotted_to_farmer=True).values("uid"
                                ).annotate(total_warehouse = Count("id")).values("total_warehouse"),
                           
                        total_galas = Gala.objects.filter(warehouse__uid = OuterRef("uid"),is_allotted_to_farmer = True).values("warehouse__uid"
                                            ).annotate(total_galas = Count("id")).values("total_galas")

)

In [12]:
farmer_qs

<QuerySet [{'property_name': 'warehouse 2', 'total_warehouse': None, 'total_galas': None}, {'property_name': 'warehouse 1', 'total_warehouse': 1, 'total_galas': 2}]>

In [13]:
Farmer.objects.get(user_uid="5f82381f-d95b-4cff-b2b8-0077c6d1f104").belong_to.get(name="Omkar").get_properties.filter(is_allotted_to_farmer=True)

<QuerySet [<Property: warehouse 1>]>

In [14]:
Farmer.objects.filter(belong_to__name = "Omkar").values("email","user_uid","username","farmer_contract__user__username","farmer_contract__warehouse__property_name")

<QuerySet [{'email': 'ayansheikh@gmail.com', 'user_uid': UUID('5f82381f-d95b-4cff-b2b8-0077c6d1f104'), 'username': 'ayan', 'farmer_contract__user__username': 'ayan', 'farmer_contract__warehouse__property_name': 'warehouse 1'}, {'email': 'deepakverma@gmail.com', 'user_uid': UUID('e73afc9f-83cf-4fa3-a28f-2d68cb2745b0'), 'username': 'deepak', 'farmer_contract__user__username': None, 'farmer_contract__warehouse__property_name': None}]>

In [15]:
from django.db.models import Value
from django.db.models.aggregates import Aggregate

class GroupConcat(Aggregate):
    function = 'GROUP_CONCAT'
    template = '%(function)s(%(distinct)s %(expressions)s)'
    allow_distinct = True

    def __init__(self, expression, delimiter=None, **extra):
        if delimiter is not None:
            self.allow_distinct = False
            delimiter_expr = Value(str(delimiter))
            super().__init__(expression, delimiter_expr, **extra)        
        else:
            super().__init__(expression, **extra)        

    def as_sqlite(self, compiler, connection, **extra_context):
        return super().as_sql(
            compiler, connection,
            function=self.function,
            template=self.template,
            **extra_context
        )

In [16]:
ayan = Farmer.objects.filter(belong_to__name = "Omkar").values("user_uid","username"
                                                       
    ).annotate(
        warehouse_name = ContractFarmer.objects.filter(user__user_uid = OuterRef("user_uid")).values("warehouse__property_name"),
        warehouseUID = ContractFarmer.objects.filter(user__user_uid = OuterRef("user_uid")).values("warehouse__uid")
                
    ).annotate(
        gala_count = Gala.objects.filter(warehouse__uid = OuterRef("warehouseUID"),is_allotted_to_farmer=True).values("warehouse__uid"
                                    ).annotate(gala_count=Count("id")).values("gala_count"),
                  
                  )









In [17]:
pd = read_frame(ayan)

In [18]:
pd

,user_uid,username,warehouse_name,warehouseUID,gala_count
0,5f82381f-d95b-4cff-b2b8-0077c6d1f104,ayan,warehouse 1,d20e7390-a74b-4a83-b6be-2f8f88e42e02,2.0
1,e73afc9f-83cf-4fa3-a28f-2d68cb2745b0,deepak,None,None,NaN


In [19]:
get_investor_qs = ContractInvestor.objects.filter(user__user_uid = "15262d8a-d00b-44b5-bbca-ca6a16633b40"
                            ).values("gala__gala_number",'owner__username','user__username')

In [20]:
get_investor_qs

<PolymorphicQuerySet [{'gala__gala_number': '4', 'owner__username': 'admin', 'user__username': 'mayank'}, {'gala__gala_number': '3', 'owner__username': 'admin', 'user__username': 'mayank'}]>

In [46]:
farmer_qs = Farmer.objects.filter(belong_to__name = "Omkar").annotate(
                        warehouse_name = ContractFarmer.objects.filter(user__user_uid = OuterRef("user_uid")).values("warehouse__property_name"),
                        warehouseUID = ContractFarmer.objects.filter(user__user_uid = OuterRef("user_uid")).values("warehouse__uid")
                    ).annotate(
                        gala_count = Gala.objects.filter(warehouse__uid = OuterRef("warehouseUID"),is_allotted_to_farmer=True).values("warehouse__uid"
                                                    ).annotate(gala_count=Count("id")).values("gala_count"),
                        remaining_gala_count = Gala.objects.filter(warehouse__uid=OuterRef("warehouseUID"),
                                                                   is_allotted_to_farmer=True,
                                                                   is_allotted=False,
                                                                   is_allotted_to_rental=False
                                                                  ).values("warehouse__uid").annotate(remaining_gala_count=Count("id")).values("remaining_gala_count"),
                        allotted_gala_count = Gala.objects.filter(warehouse__uid=OuterRef("warehouseUID"),
                                                                  is_allotted_to_farmer=True,
                                                                  is_allotted_to_rental=True,
                                                                  is_allotted=False
                                                                 ).values("warehouse__uid"
                                                                         ).annotate(allotted_gala_count=Count("id")).values("allotted_gala_count")
    
    
                                )

In [47]:
pd = read_frame(farmer_qs)

In [48]:
pd

,id,password,last_login,is_superuser,is_staff,is_active,date_joined,username,user_uid,email,...,zip_code,birth_date,app_type,created_at,updated_at,warehouse_name,warehouseUID,gala_count,remaining_gala_count,allotted_gala_count
0,5,pbkdf2_sha256$390000$9gl5aL3lTtN4z4Fi9kFaFP$3R...,None,False,False,True,2023-01-30 07:31:37.672776+00:00,ayan,5f82381f-d95b-4cff-b2b8-0077c6d1f104,ayansheikh@gmail.com,...,,1995-12-14,android,2023-01-30 07:31:37.672776+00:00,2023-01-30 07:31:37.929777+00:00,warehouse 1,d20e7390-a74b-4a83-b6be-2f8f88e42e02,2.0,1.0,1.0
1,7,pbkdf2_sha256$390000$WSCn2hwdfEuP9TzhrFZnME$9T...,None,False,False,True,2023-01-30 09:29:49.614637+00:00,deepak,e73afc9f-83cf-4fa3-a28f-2d68cb2745b0,deepakverma@gmail.com,...,,1995-12-14,android,2023-01-30 09:29:49.614637+00:00,2023-01-30 09:29:49.885475+00:00,None,None,NaN,NaN,NaN


In [68]:
farmer_property_qs = ContractFarmer.objects.filter(user__user_uid = "5f82381f-d95b-4cff-b2b8-0077c6d1f104"
                                    ).values("warehouse__uid")



In [69]:
farmer_property_qs

<PolymorphicQuerySet [{'warehouse__uid': UUID('71e0ac22-d002-4e35-a10a-bdd176531348')}, {'warehouse__uid': UUID('d20e7390-a74b-4a83-b6be-2f8f88e42e02')}]>